In [12]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy.optimize import minimize

Load Data

In [45]:
# Load drafted players list
with open('mls_2025_drafted_players.txt', 'r') as file:
    mls_2025_drafted_players = [line.strip() for line in file]

# Load player and team data
player_df = pd.read_csv('d1_player_stats.csv')
team_df = pd.read_csv('ncaa_ratings.csv')

# Clean up data
player_df = player_df.drop(columns=['Unnamed: 0'])
team_df = team_df.drop(columns=['Unnamed: 0'])
player_df = player_df[player_df['Position'] != 'Goalkeeper']
player_df = player_df[player_df['Position'] != 'Defender']


# Merge player and team data
merged_df = player_df.merge(team_df, on='Team', suffixes=('_player', '_team'))


Data Normalization

In [47]:
# Normalize statistics
merged_df['Norm_Goals'] = merged_df['Goals'] / merged_df['Goals'].max()
merged_df['Norm_Assists'] = merged_df['Assists'] / merged_df['Assists'].max()
merged_df['Norm_Shots'] = merged_df['Shots'] / merged_df['Shots'].max()
merged_df['Norm_Fouls_Won'] = merged_df.get('Fouls Won', 0) / merged_df.get('Fouls Won', 1).max()
merged_df['Norm_Minutes_Played'] = merged_df['Minutes Played'] / merged_df['Minutes Played'].max()
merged_df['Norm_ATT'] = merged_df['ATT'] / merged_df['ATT'].max()
merged_df['Norm_DEF'] = merged_df['DEF'] / merged_df['DEF'].max()

# Calculate the percentage of team minutes played
team_total_minutes = merged_df.groupby('Team')['Minutes Played'].transform('sum')
merged_df['Team_Minutes_Played_Percentage'] = merged_df['Minutes Played'] / team_total_minutes


Objective Function and Optimization

In [49]:
# Initial weights and constraints
initial_weights = [
    0.3, 0.2, 0.1, 0.25, 0.15,  # Forward weights
    0.15, 0.25, 0.05, 0.3, 0.25,  # Midfielder weights
    0.05, 0.1, 0.05, 0.6, 0.2   # Defender weights
]
bounds = [(0, 1)] * len(initial_weights)
constraints = [
    {'type': 'eq', 'fun': lambda w: sum(w[0:5]) - 1},  # Forward
    {'type': 'eq', 'fun': lambda w: sum(w[5:10]) - 1},  # Midfielder
    {'type': 'eq', 'fun': lambda w: sum(w[10:15]) - 1}  # Defender
]

# Objective function
def objective(weights):
    updated_weights = {
        'Forward': {'Goals': weights[0], 'Assists': weights[1], 'Shots': weights[2], 'Team_Att': weights[3], 'Fouls_Won': weights[4]},
        'Midfielder': {'Goals': weights[5], 'Assists': weights[6], 'Shots': weights[7], 'Team_Att_Def': weights[8], 'Fouls_Won': weights[9]},
        'Defender': {'Goals': weights[10], 'Assists': weights[11], 'Shots': weights[12], 'Team_Def': weights[13], 'Fouls_Won': weights[14]}
    }

    def calculate_rating(row):
        if row['Position'] == 'Forward':
            return (
                row['Norm_Goals'] * updated_weights['Forward']['Goals'] +
                row['Norm_Assists'] * updated_weights['Forward']['Assists'] +
                row['Norm_Shots'] * updated_weights['Forward']['Shots'] +
                row['Norm_Fouls_Won'] * updated_weights['Forward']['Fouls_Won'] +
                row['Norm_ATT'] * row['Team_Minutes_Played_Percentage'] * updated_weights['Forward']['Team_Att']
            ) * 100
        elif row['Position'] == 'Midfielder':
            return (
                row['Norm_Goals'] * updated_weights['Midfielder']['Goals'] +
                row['Norm_Assists'] * updated_weights['Midfielder']['Assists'] +
                row['Norm_Shots'] * updated_weights['Midfielder']['Shots'] +
                row['Norm_Fouls_Won'] * updated_weights['Midfielder']['Fouls_Won'] +
                ((row['Norm_ATT'] + row['Norm_DEF']) / 2) * row['Team_Minutes_Played_Percentage'] * updated_weights['Midfielder']['Team_Att_Def']
            ) * 100
        elif row['Position'] == 'Defender':
            return (
                row['Norm_Goals'] * updated_weights['Defender']['Goals'] +
                row['Norm_Assists'] * updated_weights['Defender']['Assists'] +
                row['Norm_Shots'] * updated_weights['Defender']['Shots'] +
                row['Norm_Fouls_Won'] * updated_weights['Defender']['Fouls_Won'] +
                row['Norm_DEF'] * row['Team_Minutes_Played_Percentage'] * updated_weights['Defender']['Team_Def']
            ) * 100
        else:
            return 0

    merged_df['Overall_Rating'] = merged_df.apply(calculate_rating, axis=1)
    df = merged_df.copy()
    df['MAX'] = ((df['Overall_Rating'] - df['Overall_Rating'].min()) /
                 (df['Overall_Rating'].max() - df['Overall_Rating'].min()) * 100).fillna(0)
    drafted_players_df = df[df['Name'].isin(mls_2025_drafted_players)]
    return -drafted_players_df['MAX'].mean()

# Optimize weights
result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints, method='SLSQP')
optimized_weights = result.x
print("Optimized Weights:", optimized_weights)


Optimized Weights: [1.97299529e-02 5.47487220e-16 5.58439402e-02 9.03878403e-01
 2.05477044e-02 5.65730703e-03 7.40706787e-16 3.91778613e-02
 8.47183184e-01 1.07981648e-01 4.99999972e-02 9.99999972e-02
 4.99999849e-02 6.00000011e-01 2.00000010e-01]


Calculate and Scale the Ratings

In [50]:
# Organize optimized weights into a dictionary
normalized_values = optimized_weights  # Get optimized weights from the minimize() result

weights = {
    'Forward': {
        'Goals': normalized_values[0],
        'Assists': normalized_values[1],
        'Shots': normalized_values[2],
        'Team_Att': normalized_values[3],
        'Fouls_Won': normalized_values[4]
    },
    'Midfielder': {
        'Goals': normalized_values[5],
        'Assists': normalized_values[6],
        'Shots': normalized_values[7],
        'Team_Att_Def': normalized_values[8],
        'Fouls_Won': normalized_values[9]
    },
    'Defender': {
        'Goals': normalized_values[10],
        'Assists': normalized_values[11],
        'Shots': normalized_values[12],
        'Team_Def': normalized_values[13],
        'Fouls_Won': normalized_values[14]
    }
}

# Define function to calculate player rating based on position
def calculate_rating(row):
    if row['Position'] == 'Forward':
        return (
            row['Norm_Goals'] * weights['Forward']['Goals'] +
            row['Norm_Assists'] * weights['Forward']['Assists'] +
            row['Norm_Shots'] * weights['Forward']['Shots'] +
            row['Norm_Fouls_Won'] * weights['Forward']['Fouls_Won'] +
            row['Norm_ATT'] * row['Team_Minutes_Played_Percentage'] * weights['Forward']['Team_Att']
        ) * 100
    elif row['Position'] == 'Midfielder':
        return (
            row['Norm_Goals'] * weights['Midfielder']['Goals'] +
            row['Norm_Assists'] * weights['Midfielder']['Assists'] +
            row['Norm_Shots'] * weights['Midfielder']['Shots'] +
            row['Norm_Fouls_Won'] * weights['Midfielder']['Fouls_Won'] +
            ((row['Norm_ATT'] + row['Norm_DEF']) / 2) * row['Team_Minutes_Played_Percentage'] * weights['Midfielder']['Team_Att_Def']
        ) * 100
    elif row['Position'] == 'Defender':
        return (
            row['Norm_Goals'] * weights['Defender']['Goals'] +
            row['Norm_Assists'] * weights['Defender']['Assists'] +
            row['Norm_Shots'] * weights['Defender']['Shots'] +
            row['Norm_Fouls_Won'] * weights['Defender']['Fouls_Won'] +
            row['Norm_DEF'] * row['Team_Minutes_Played_Percentage'] * weights['Defender']['Team_Def']
        ) * 100
    else:
        return 0  # Default for unknown positions


# Apply the rating calculation
merged_df['Overall_Rating'] = merged_df.apply(calculate_rating, axis=1)

# Apply logarithmic scaling to compress rating range
merged_df['Log_Rating'] = np.log1p(merged_df['Overall_Rating'])

# Normalize ratings to a 0-100 range
log_rating_min = merged_df['Log_Rating'].min()
log_rating_max = merged_df['Log_Rating'].max()
merged_df['MAX'] = (
    (merged_df['Log_Rating'] - log_rating_min) / (log_rating_max - log_rating_min) * 100
).astype(int)


Save the data

In [51]:
df = merged_df[['Name', 'Team', 'Position', 'Goals', 'Assists', 'Shots', 'Fouls Won', 
                'Minutes Played', 'Overall_Rating', 'MAX']]
df = df.sort_values(by='MAX', ascending=False).reset_index(drop=True)
df['Rank'] = df.index + 1  # Create a Rank column starting from 1
#df.to_csv(r"PlayerRatings.csv", index=False)


In [52]:
df.head(20)

,Name,Team,Position,Goals,Assists,Shots,Fouls Won,Minutes Played,Overall_Rating,MAX,Rank
0,Steven Cordova,Presbyterian,Midfielder,1,6,17,44,1409,17.068507,100,1
1,Elie Kisoka,Kansas City,Forward,11,9,67,29,1684,17.068328,99,2
2,Michael Adedokun,Ohio St.,Midfielder,11,11,83,33,1855,17.068489,99,3
3,Emil Jaaskelainen,Akron,Forward,23,7,92,52,1702,17.068504,99,4
4,Bailey Sparks,SMU,Forward,9,10,49,29,1804,17.068432,99,5
5,Carson Hodgson,Western Mich.,Midfielder,6,6,38,57,1747,17.068220,99,6
6,Nikola Djordjevic,SMU,Midfielder,1,5,24,48,1590,16.262720,98,7
7,Alex Harris,Cornell,Forward,19,5,86,20,1464,16.346282,98,8
8,Grant Paskus,NIU,Midfielder,1,3,10,58,1184,16.519868,98,9
9,Albin Gashi,Virginia,Midfielder,2,2,22,55,1678,15.974799,97,10
